In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("item_list_to_redis") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/22 10:22:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/22 10:22:59 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
import redis
def send_hot_list(item):
    pool = redis.ConnectionPool(host='cowstudio.wayne-lee.cn',port=3002,password='cowstudio', decode_responses=True)
    redis_cli = redis.Redis(connection_pool=pool)
    item_id,category,score,rn,date = item
    redis_cli.zadd("item-hot-list-"+category,{item_id:score})
    return f"item-hot-list-{category}",item_id,score
def send_fresh_list(item):
    pool = redis.ConnectionPool(host='cowstudio.wayne-lee.cn',port=3002,password='cowstudio', decode_responses=True)
    redis_cli = redis.Redis(connection_pool=pool)
    item_id,category,score,rn,date = item
    redis_cli.zadd("item-fresh-list-"+category,{item_id:score})
    return f"item-fresh-list-{category}",item_id,score

In [3]:
# define map functions 
from datetime import datetime, timedelta

today_string = datetime.today().strftime('%Y-%m-%d')
print(today_string)

2023-04-22


In [4]:
item_hot_list = spark.sql(f'''
select
    *
from
    item_hot_list
where
    date = '{today_string}'
''').rdd

print(item_hot_list.take(1))
item_hot_list.map(send_hot_list).collect()

23/04/22 10:23:14 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


[Row(item_id=83, category='cattle_product', score=10.0, rn=13, date='2023-04-22')]


[('item-hot-list-cattle_product', 83, 10.0),
 ('item-hot-list-breeding', 198, 14.0),
 ('item-hot-list-service_product', 221, 20.0),
 ('item-hot-list-service_product', 455, 35.0),
 ('item-hot-list-service', 457, 20.0),
 ('item-hot-list-cattle_product', 70, 23.0),
 ('item-hot-list-cattle_product', 95, 68.0),
 ('item-hot-list-cattle_product', 78, 32.0),
 ('item-hot-list-cattle_product', 100, 30.0),
 ('item-hot-list-cattle_product', 41, 20.0),
 ('item-hot-list-cattle_product', 80, 13.0),
 ('item-hot-list-cattle_product', 87, 12.0),
 ('item-hot-list-whole_cattle', 359, 25.0),
 ('item-hot-list-service', 449, 34.0),
 ('item-hot-list-cattle_product', 77, 37.0),
 ('item-hot-list-breeding', 191, 10.0),
 ('item-hot-list-breeding', 197, 10.0),
 ('item-hot-list-cattle_product', 81, 11.0),
 ('item-hot-list-service_product', 212, 16.0),
 ('item-hot-list-service_product', 228, 15.0),
 ('item-hot-list-whole_cattle', 364, 34.0),
 ('item-hot-list-service_product', 232, 20.0),
 ('item-hot-list-cattle_prod

In [5]:
item_fresh_list = spark.sql(f'''
select
    *
from
    item_fresh_list
where
    date = '{today_string}'
''').rdd

print(item_fresh_list.take(1))
item_fresh_list.map(send_fresh_list).collect().task(10)

[Row(item_id=121, category='whole_cattle', score=7.0, rn=3, date='2023-04-22')]


[('item-fresh-list-whole_cattle', 121, 7.0),
 ('item-fresh-list-breeding', 203, 8.0),
 ('item-fresh-list-whole_cattle', 355, 7.0),
 ('item-fresh-list-breeding', 196, 8.0),
 ('item-fresh-list-whole_cattle', 361, 7.0),
 ('item-fresh-list-breeding', 194, 5.0),
 ('item-fresh-list-service', 452, 4.0),
 ('item-fresh-list-whole_cattle', 403, 6.0),
 ('item-fresh-list-service_product', 257, 8.0),
 ('item-fresh-list-breeding', 372, 4.0),
 ('item-fresh-list-service', 389, 4.0),
 ('item-fresh-list-whole_cattle', 414, 5.0),
 ('item-fresh-list-service_product', 223, 8.0),
 ('item-fresh-list-breeding', 195, 2.0),
 ('item-fresh-list-service', 456, 4.0),
 ('item-fresh-list-whole_cattle', 127, 4.0),
 ('item-fresh-list-service_product', 220, 6.0),
 ('item-fresh-list-breeding', 193, 2.0),
 ('item-fresh-list-service', 298, 2.0),
 ('item-fresh-list-whole_cattle', 369, 4.0),
 ('item-fresh-list-service_product', 230, 5.0),
 ('item-fresh-list-breeding', 192, 1.0),
 ('item-fresh-list-service', 448, 2.0),
 ('ite

In [6]:
spark.stop()